In [68]:
import pandas as pd
import numpy as np

# Tải dữ liệu

df_stats = pd.read_csv("//content/players_stats.csv")
df_transfers = pd.read_csv("/content/player_transfers.csv")

print("--- Thông tin tệp Stats (5 hàng đầu) ---")
print(df_stats.head())
print("\n--- Thông tin tệp Stats (Info) ---")
df_stats.info()

print("\n--- Thông tin tệp Transfers (5 hàng đầu) ---")
print(df_transfers.head())
print("\n--- Thông tin tệp Transfers (Info) ---")
df_transfers.info()

# Thực hiện gộp (merge)
# Chúng ta dùng 'left' merge để giữ tất cả cầu thủ từ df_stats
# Chúng ta liên kết 'Name' (từ stats) với 'player_name' (từ transfers)
df_merged = pd.merge(
    df_stats,
    df_transfers,
    left_on='Name',
    right_on='player_name',
    how='left',
    suffixes=('_stats', '_transfer') # Thêm hậu tố để phân biệt các cột trùng tên (ví dụ: Team)
)

# Dọn dẹp các cột thừa sau khi gộp
# Chúng ta muốn giữ cột 'Team' từ tệp stats (bây giờ là 'Team_stats')
# và loại bỏ 'player_id', 'player_name', và 'team_transfer'
cols_to_drop = ['player_id', 'player_name', 'team_transfer']
# Đảm bảo chỉ loại bỏ các cột thực sự tồn tại
existing_cols_to_drop = [col for col in cols_to_drop if col in df_merged.columns]
df_merged = df_merged.drop(columns=existing_cols_to_drop)

# Đổi tên cột 'Team_stats' (nếu tồn tại) trở lại 'Team'
if 'Team_stats' in df_merged.columns:
    df_merged = df_merged.rename(columns={'Team_stats': 'Team'})

print("\n--- DataFrame đã gộp (5 hàng đầu) ---")
print(df_merged.head())

print("\n--- DataFrame đã gộp (Info) ---")
df_merged.info()

# Lưu kết quả
output_filename = "players_stats_with_transfers.csv"
df_merged.to_csv(output_filename, index=False)

print(f"\nĐã gộp dữ liệu thành công và lưu vào tệp: {output_filename}")

--- Thông tin tệp Stats (5 hàng đầu) ---
                Name   Nation         Team Position  Age  Matches_Played  \
0  Joshua Acheampong  eng ENG      Chelsea       DF   18               4   
1        Tyler Adams   us USA  Bournemouth       MF   25              28   
2   Tosin Adarabioyo  eng ENG      Chelsea       DF   26              22   
3      Simon Adingra   ci CIV     Brighton     FWMF   22              29   
4   Emmanuel Agbadou   ci CIV       Wolves       DF   27              16   

   Starts  Minutes  Goals  Assists  ...  Dispossessed  Passes_Received  \
0       2      170      0        0  ...             0              111   
1      21     1965      0        3  ...            12              757   
2      15     1409      1        1  ...             0             1000   
3      12     1097      2        2  ...            12              388   
4      16     1410      1        0  ...             4              796   

   Fouls_Committed  Fouls_Drawn  Offsides  Crosses  Ball_

In [69]:
import pandas as pd
import numpy as np

# Tên tệp từ bước trước
input_filename = "/content/players_stats_with_transfers.csv"
output_filename = "players_stats_cleaned.csv"

try:
    df = pd.read_csv(input_filename)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp '{input_filename}'.")
    # exit()

print(f"--- Đã tải tệp '{input_filename}' ---")

# Các cột được xác định là 'object' (văn bản) trong bước trước
# mà có thể chứa giá trị 'N/a'
cols_with_na = [
    'GA90', 'Save_Pct', 'CS_Pct', 'PK_Save_Pct', 'SoT_Pct',
    'Goals_Per_Shot', 'Avg_Shot_Distance', 'Long_Pass_Pct',
    'Take_Ons_Success_Pct', 'Take_Ons_Tackled_Pct', 'Aerials_Won_Pct'
]

print("\n--- Kiểu dữ liệu (Dtypes) TRƯỚC khi thay thế 'N/a' ---")
# Lọc ra các cột thực sự tồn tại trong DataFrame để tránh lỗi
existing_cols = [col for col in cols_with_na if col in df.columns]
print(df[existing_cols].dtypes)

# 1. Thay thế tất cả các giá trị 'N/a' (và 'N/A' để chắc chắn) bằng 0
df.replace(['N/a', 'N/A'], 0, inplace=True)

# 2. Chuyển đổi các cột này sang kiểu số (numeric)
# Điều này rất quan trọng để có thể thực hiện các phép tính sau này
for col in existing_cols:
    # Sau khi thay thế 'N/a' bằng 0, pd.to_numeric sẽ hoạt động
    df[col] = pd.to_numeric(df[col])

print("\n--- Kiểu dữ liệu (Dtypes) SAU khi thay thế và chuyển đổi ---")
print(df[existing_cols].dtypes)

# Kiểm tra 5 hàng đầu của một vài cột đã thay đổi
print("\n--- 5 hàng đầu của các cột đã chuyển đổi ---")
print(df[['Name', 'GA90', 'Save_Pct', 'Aerials_Won_Pct']].head())

# 3. Lưu DataFrame đã dọn dẹp vào một tệp CSV mới
df.to_csv(output_filename, index=False)

print(f"\nĐã thay thế 'N/a' bằng 0 và lưu kết quả vào: {output_filename}")

--- Đã tải tệp '/content/players_stats_with_transfers.csv' ---

--- Kiểu dữ liệu (Dtypes) TRƯỚC khi thay thế 'N/a' ---
GA90                    object
Save_Pct                object
CS_Pct                  object
PK_Save_Pct             object
SoT_Pct                 object
Goals_Per_Shot          object
Avg_Shot_Distance       object
Long_Pass_Pct           object
Take_Ons_Success_Pct    object
Take_Ons_Tackled_Pct    object
Aerials_Won_Pct         object
dtype: object

--- Kiểu dữ liệu (Dtypes) SAU khi thay thế và chuyển đổi ---
GA90                    float64
Save_Pct                float64
CS_Pct                  float64
PK_Save_Pct             float64
SoT_Pct                 float64
Goals_Per_Shot          float64
Avg_Shot_Distance       float64
Long_Pass_Pct           float64
Take_Ons_Success_Pct    float64
Take_Ons_Tackled_Pct    float64
Aerials_Won_Pct         float64
dtype: object

--- 5 hàng đầu của các cột đã chuyển đổi ---
                Name  GA90  Save_Pct  Aerials_Won_Pc

In [70]:
import pandas as pd
import numpy as np

# Chúng ta phải bắt đầu từ tệp 'cleaned', vì tệp 'numeric_only' đã làm mất cột 'transfer_value'
input_file = "/content/players_stats_cleaned.csv"
output_file = "players_stats_for_model.csv"

try:
    df = pd.read_csv(input_file)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp '{input_file}'.")
    # exit()

print(f"--- Đã tải tệp '{input_file}' ---")
print("--- Cột 'transfer_value' TRƯỚC KHI xử lý ---")
print(df['transfer_value'].head())

# 1. Định nghĩa hàm để chuyển đổi giá trị (ví dụ: "€1.4M")
def convert_transfer_value(value):
    # Nếu giá trị là NaN (từ left merge) hoặc không phải là string, trả về NaN
    if pd.isna(value) or not isinstance(value, str):
        return np.nan

    value = value.replace('€', '') # Bỏ ký hiệu €

    if 'M' in value:
        # Nếu là 'M' (Triệu), nhân với 1,000,000
        return float(value.replace('M', '')) * 1_000_000
    elif 'k' in value:
        # Nếu là 'k' (Ngàn), nhân với 1,000
        return float(value.replace('k', '')) * 1_000
    else:
        # Nếu không có M hoặc k
        try:
            return float(value)
        except ValueError:
            return np.nan # Trả về NaN nếu không thể chuyển đổi

# 2. Áp dụng hàm này cho cột 'transfer_value'
# Chúng ta sẽ ghi đè trực tiếp lên cột cũ
df['transfer_value'] = df['transfer_value'].apply(convert_transfer_value)

# Đổi tên cột cho rõ ràng
df.rename(columns={'transfer_value': 'transfer_value_numeric'}, inplace=True)

print("\n--- Cột 'transfer_value_numeric' SAU KHI xử lý ---")
print(df['transfer_value_numeric'].head())

# 3. Bây giờ, loại bỏ TẤT CẢ các cột object còn lại
print("\n--- Thông tin DataFrame (TRƯỚC KHI loại bỏ object) ---")
# df.info() sẽ hiển thị tất cả các cột, chúng ta có thể thấy các cột object
# (Name, Nation, Team, Position, team) và cột 'transfer_value_numeric' (bây giờ là float)
object_cols = df.select_dtypes(include=['object']).columns
print(f"\n--- Các cột 'object' sẽ bị loại bỏ ---")
print(list(object_cols))

df_model = df.drop(columns=object_cols)

print("\n--- Thông tin DataFrame (SAU KHI loại bỏ object) ---")
df_model.info()

# 4. Lưu tệp
df_model.to_csv(output_file, index=False)
print(f"\nĐã xử lý 'transfer_value' và loại bỏ các cột object. Đã lưu vào {output_file}")

--- Đã tải tệp '/content/players_stats_cleaned.csv' ---
--- Cột 'transfer_value' TRƯỚC KHI xử lý ---
0     €1.4M
1    €24.5M
2    €24.8M
3      €34M
4      €18M
Name: transfer_value, dtype: object

--- Cột 'transfer_value_numeric' SAU KHI xử lý ---
0     1400000.0
1    24500000.0
2    24800000.0
3    34000000.0
4    18000000.0
Name: transfer_value_numeric, dtype: float64

--- Thông tin DataFrame (TRƯỚC KHI loại bỏ object) ---

--- Các cột 'object' sẽ bị loại bỏ ---
['Name', 'Nation', 'Team', 'Position', 'team']

--- Thông tin DataFrame (SAU KHI loại bỏ object) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Data columns (total 72 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           516 non-null    int64  
 1   Matches_Played                516 non-null    int64  
 2   Starts                        516 non-null    int64  
 3   Minutes                 

In [71]:
df = pd.read_csv("/content/players_stats_for_model.csv")

In [72]:
print(df.info)

<bound method DataFrame.info of      Age  Matches_Played  Starts  Minutes  Goals  Assists  Yellow_Cards  \
0     18               4       2      170      0        0             1   
1     25              28      21     1965      0        3             7   
2     26              22      15     1409      1        1             4   
3     22              29      12     1097      2        2             0   
4     27              16      16     1410      1        0             3   
..   ...             ...     ...      ...    ...      ...           ...   
511   39              32      19     1875      1        3             7   
512   21              36      35     3109      0        1             4   
513   27              15       5      527      0        1             1   
514   23              32      14     1402      3        1             2   
515   25              30      26     2325      3        8             4   

     Red_Cards   xG  xAG  ...  Passes_Received  Fouls_Committed  Fo

In [73]:
import pandas as pd
import numpy as np

# Tên tệp
input_file = "players_stats_for_model.csv"

try:
    df = pd.read_csv(input_file)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp '{input_file}'.")
    # exit()

print(f"--- Đã tải tệp '{input_file}' ---")

# --- Xử lý cột 'transfer_value_numeric' ---

# Kiểm tra xem có bao nhiêu giá trị null
null_values = df['transfer_value_numeric'].isnull().sum()
print(f"Số lượng giá trị null (trống) trong 'transfer_value_numeric': {null_values}")

# Loại bỏ các hàng bị null/trống để tính toán chính xác
df_cleaned = df.dropna(subset=['transfer_value_numeric'])

print(f"Số lượng cầu thủ được tính toán: {len(df_cleaned)}")

# --- Tính toán ---
mean_value = df_cleaned['transfer_value_numeric'].mean()
median_value = df_cleaned['transfer_value_numeric'].median()

print("\n--- Kết quả ---")
print(f"Giá trung bình (Mean) của cầu thủ: €{mean_value:,.2f}")
print(f"Giá trung vị (Median) của cầu thủ:  €{median_value:,.2f}")

--- Đã tải tệp 'players_stats_for_model.csv' ---
Số lượng giá trị null (trống) trong 'transfer_value_numeric': 2
Số lượng cầu thủ được tính toán: 514

--- Kết quả ---
Giá trung bình (Mean) của cầu thủ: €25,087,159.53
Giá trung vị (Median) của cầu thủ:  €19,300,000.00


In [82]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# --- 1. Tải và chuẩn bị dữ liệu ---
input_file = "players_stats_for_model.csv"

try:
    df = pd.read_csv(input_file)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp '{input_file}'.")
    # exit()
df = df[df['transfer_value_numeric'] > 0]
# Loại bỏ các hàng thiếu giá trị ở cột target
df = df.dropna(subset=['transfer_value_numeric'])

# Xác định X (features) và y (target)
X = df.drop(columns=['transfer_value_numeric'])
y = df['transfer_value_numeric']

feature_names = X.columns.tolist()
print(f"Đã tải dữ liệu. Số lượng features: {len(feature_names)}, số mẫu huấn luyện: {len(df)}")

# --- 2. Chia dữ liệu ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# --- 3. Định nghĩa và Huấn luyện mô hình ---
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=6,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.2,
    gamma=0.2,
    random_state=42,
    tree_method='hist',
    n_jobs=-1
)

print("\n--- Bắt đầu huấn luyện XGBoost ---")
xgb_model.fit(X_train, y_train)
print("--- Huấn luyện hoàn tất ---")

# --- 4. Đánh giá mô hình ---
y_pred = xgb_model.predict(X_test)

# Các chỉ số đánh giá
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # đổi sang %

print("\n--- Kết quả đánh giá trên tập Test (XGBoost) ---")
print(f"RMSE (Sai số trung bình): ${rmse:,.2f}")
print(f"R-squared (R^2): {r2:.4f}")
print(f"MAPE (Sai số phần trăm tuyệt đối trung bình): {mape:.2f}%")

# --- 5. Các features quan trọng ---
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("\n--- 10 FEATURES QUAN TRỌNG NHẤT ---")
print(importance_df.head(10))

# --- 6. So sánh dự đoán ---
print("\n--- So sánh dự đoán (5 mẫu đầu tiên) ---")
comparison_df = pd.DataFrame({
    'Actual Value': y_test.values[:5],
    'Predicted Value': y_pred[:5]
})
print(comparison_df)


Đã tải dữ liệu. Số lượng features: 71, số mẫu huấn luyện: 505

--- Bắt đầu huấn luyện XGBoost ---
--- Huấn luyện hoàn tất ---

--- Kết quả đánh giá trên tập Test (XGBoost) ---
RMSE (Sai số trung bình): $13,767,130.94
R-squared (R^2): 0.7110
MAPE (Sai số phần trăm tuyệt đối trung bình): 192.27%

--- 10 FEATURES QUAN TRỌNG NHẤT ---
                      Feature  Importance
35                        SCA    0.188631
51            Touches_Att_3rd    0.087736
37                        GCA    0.067571
11         Progressive_Passes    0.057183
0                         Age    0.039972
50            Touches_Mid_3rd    0.027448
34  Crosses_Into_Penalty_Area    0.022329
52            Touches_Att_Pen    0.019941
32    Passes_Into_Final_Third    0.019339
56                    Carries    0.018366

--- So sánh dự đoán (5 mẫu đầu tiên) ---
   Actual Value  Predicted Value
0    20700000.0       16111548.0
1    23800000.0       26127642.0
2     7300000.0        3925885.0
3    19600000.0       22449130.0